In [3]:
import pandas as pd
import numpy as np
import os

prepare_RSgroups = False #надо ли объединить группы в один файл


dataset_name = 'prepared_facebook_data'
folder_name = 'facebook_comm_vol'
DIR_WITH_FILES= '../other_data_experiments/{}/'.format(folder_name)

DIR_TO_SAVE_RES = DIR_WITH_FILES

In [4]:
if prepare_RSgroups:
    for i in range(1,11):
        part_1 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group1_{}_{}.csv".format(dataset_name, i))
        part_2 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group2_{}_{}.csv".format(dataset_name, i))
        part_3 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group3_{}_{}.csv".format(dataset_name, i))
        full = pd.concat([part_1,part_2,part_3], sort = False)
        full['run_number']=i
        print(full.shape)
        full.to_csv(DIR_TO_SAVE_RES+"res_rand_groups_{}_{}.csv".format(dataset_name, i), index=False)

In [5]:
RS_time = np.array([])
print(np.mean(RS_time))
print(np.std(RS_time))

nan
nan


D:\Games\conda\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Games\conda\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Games\conda\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
D:\Games\conda\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
D:\Games\conda\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
group_1_time = np.array([])
group_2_time = np.array([])
group_3_time = np.array([])

sum_arr=np.sum([group_1_time, group_2_time, group_3_time], axis = 0)


for arr in [group_1_time, group_2_time, group_3_time, sum_arr]:
    print(np.mean(arr))
    print(np.std(arr))
    print("=====")

nan
nan
=====
nan
nan
=====
nan
nan
=====
nan
nan
=====


In [7]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i_copy.loc[
          :, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()

      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score
    
      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums#, best_score       
   

In [8]:
group_df = pd.DataFrame([])
for i in range(1,11):
    df = pd.read_csv(DIR_TO_SAVE_RES+"res_rand_groups_{}_{}.csv".format(dataset_name, i))    
    df['iter_num'] = range(1, df.shape[0]+1)
    group_df = pd.concat([group_df, df], sort = False)

In [9]:
stop_iter_num = 50
best_params_first_group, first_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=1, right_iter_num=243)

In [10]:
best_params_first_group

{1: "{'max_depth': 6, 'min_child_weight': 9}",
 2: "{'max_depth': 6, 'min_child_weight': 9}",
 3: "{'max_depth': 6, 'min_child_weight': 9}",
 4: "{'max_depth': 6, 'min_child_weight': 9}",
 5: "{'max_depth': 6, 'min_child_weight': 9}",
 6: "{'max_depth': 6, 'min_child_weight': 9}",
 7: "{'max_depth': 6, 'min_child_weight': 9}",
 8: "{'max_depth': 6, 'min_child_weight': 6}",
 9: "{'max_depth': 6, 'min_child_weight': 9}",
 10: "{'max_depth': 5, 'min_child_weight': 7}"}

In [11]:
first_stop_iter_nums

{1: 243,
 2: 243,
 3: 171,
 4: 243,
 5: 105,
 6: 50,
 7: 154,
 8: 243,
 9: 133,
 10: 243}

In [3]:
#import os 

In [17]:
#os.listdir("./data/tmp")

['GR_S_2_group_with_10_6_house_prices.csv',
 'GR_S_2_group_with_100_9_house_prices.csv',
 'GR_S_2_group_with_25_8_house_prices.csv']

In [13]:
stop_iter_num = 50
best_params_second_group={}
second_stop_iter_nums={}
    
for i in [1,2,4,8,10]:
    group_df = pd.read_csv(DIR_TO_SAVE_RES+"res_rand_groups_{}_{}.csv".format(dataset_name, i))
    group_df['iter_num'] = range(1, group_df.shape[0]+1)
    first_group_stop_iters = 243
    left_iter_num = first_group_stop_iters+1
    right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

    best_params_second_group_tmp, second_stop_iter_nums_tmp = group_best_params_with_stop(
        group_df, stop_iter_num = stop_iter_num,
        left_iter_num=left_iter_num, right_iter_num=right_iter_num)
    best_params_second_group.update(best_params_second_group_tmp)
    second_stop_iter_nums.update(second_stop_iter_nums_tmp)

In [14]:
print(best_params_second_group)

{1: "{'alpha': 0.7742493752904185, 'lambda': 0.8395446532215035}", 2: "{'alpha': 0.8295577863095839, 'lambda': 0.423505767258905}", 4: "{'alpha': 0.5799345256769086, 'lambda': 0.018130210650242562}", 8: "{'alpha': 0.7272678704394513, 'lambda': 0.686795003613021}", 10: "{'alpha': 0.8430393110070236, 'lambda': 0.8105824126665478}"}


In [15]:
print(second_stop_iter_nums)

{1: 293, 2: 293, 4: 293, 8: 293, 10: 293}


In [ ]:
# добавить код склейки для 3 группы со второй, если первая группа была полной 


"""
res_rand_3=res_rand_3.loc[(res_rand_3['run_number']==params_set_num) & \
                                (res_rand_3['iter_num']<=int(third_stop_iter_nums[params_set_num]))]

        res=pd.concat([res_rand_2,res_rand_3], sort=False)

        sec_idxs = res.loc[res['experiment_name']=='random search for 2 group','iter_num'].values
        third_idxs = res.loc[res['experiment_name']=='random search for 3 group','iter_num'].values
        intersection = np.intersect1d(third_idxs, sec_idxs)

        res = res.loc[~((res['experiment_name']=='random search for 2 group') & (res['iter_num'].isin(intersection)))]

        res.to_csv('GR_S_2_and_3_groups_with_{}_{}_{}.csv'.format(stop_criterion, params_set_num, dataset_name), index=False)
        print(first_group_stop_iters[params_set_num]+res.shape[0])
        print('done for third group')
    else:
        print("first group takes 243 iterations, dataset num: ", params_set_num)
"""

In [30]:
group_df = pd.read_csv("./data/tmp/GR_S_2_group_with_25_8_house_prices.csv")
# {'max_depth': 3, 'min_child_weight': 3}

stop_iter_num = 100

first_group_stop_iters = 109
left_iter_num = first_group_stop_iters+1
right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

best_params_second_group, second_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=left_iter_num, right_iter_num=right_iter_num)

In [31]:
best_params_second_group

{8: "{'alpha': 0.5391121857294692, 'lambda': 0.7030791263648584}"}

In [32]:
second_stop_iter_nums

{8: 209}